<a href="https://colab.research.google.com/github/athabrani/BTCUSD-Scraper/blob/main/Scrapping_data_BTCUSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Scrapping data dari bitstamp


In [ ]:
import pandas as pd
import requests
from math import ceil
import time


PAIR = "btcusd"
URL = f"https://www.bitstamp.net/api/v2/ohlc/{PAIR}/"

STEP = 4 * 60 * 60          # 4 jam = 14400 detik
TOTAL_NEEDED = 9000         # target jumlah candle
PER_CALL_LIMIT = 1000       # batas aman per request
TIMEOUT = 30                # detik untuk requests


all_rows = []
remaining = TOTAL_NEEDED


end_ts = int(time.time())

while remaining > 0:
    batch_limit = min(PER_CALL_LIMIT, remaining)
    params = {
        "step": STEP,
        "limit": batch_limit,
        "end": end_ts
    }
    r = requests.get(URL, params=params, timeout=TIMEOUT)
    r.raise_for_status()
    payload = r.json()["data"]["ohlc"]

    if not payload:
        print("Tidak ada data yang dikembalikan. Berhenti lebih awal.")
        break


    payload.sort(key=lambda x: int(x["timestamp"]))

    all_rows.extend(payload)


    earliest_ts = int(payload[0]["timestamp"])
    end_ts = earliest_ts - 1

    remaining -= len(payload)


df_stream = pd.DataFrame(all_rows)


df_stream.rename(columns={
    "timestamp": "Timestamp",
    "open": "Open",
    "high": "High",
    "low": "Low",
    "close": "Close",
    "volume": "Volume"
}, inplace=True)


df_stream["Timestamp"] = pd.to_datetime(
    df_stream["Timestamp"].astype(int), unit="s", utc=True
).dt.tz_convert("Asia/Jakarta").dt.tz_localize(None)


for col in ["Open", "High", "Low", "Close", "Volume"]:
    df_stream[col] = pd.to_numeric(df_stream[col], errors="coerce")


df_stream.drop_duplicates(subset=["Timestamp"], keep="first", inplace=True)

df_stream.sort_values("Timestamp", inplace=True)

if len(df_stream) > TOTAL_NEEDED:
    df_stream = df_stream.tail(TOTAL_NEEDED)


output_path = "/btcusd_bitstamp_4h_9000.csv"
df_stream.to_csv(output_path, index=False)

print(f"Data berhasil diexport ke: {output_path}")
print(df_stream.head())
print("Total baris:", len(df_stream))


Data berhasil diexport ke: /btcusd_bitstamp_4h_9000.csv
               Timestamp      Open      High       Low     Close      Volume
8000 2021-09-29 23:00:00  41687.48  41807.10  41038.49  41194.33  328.773574
8001 2021-09-30 03:00:00  41175.95  41566.37  40900.00  41542.90  184.691766
8002 2021-09-30 07:00:00  41537.56  43731.58  41427.87  43580.52  374.906749
8003 2021-09-30 11:00:00  43621.31  43835.09  43225.00  43290.72  440.090322
8004 2021-09-30 15:00:00  43294.61  43460.00  42725.00  43086.49  439.272284
Total baris: 9000


#Data Historis

menghapus 30% dari data historis

In [ ]:
import pandas as pd
import os

input_path = "/btc_4h.csv"
output_path = "/btc_4h_minus_last9000.csv"

df = pd.read_csv(input_path)

before_len = len(df)
drop_count = 9000 if before_len >= 9000 else before_len
after_len = before_len - drop_count

df_trim = df.iloc[:after_len]

df_trim.to_csv(output_path, index=False)

stats = {
    "rows_before": before_len,
    "rows_dropped": drop_count,
    "rows_after": after_len,
    "columns": list(df.columns),
}

if "Timestamp" in df_trim.columns and not df_trim.empty:
    stats["timestamp_min"] = str(df_trim["Timestamp"].iloc[0])
    stats["timestamp_max"] = str(df_trim["Timestamp"].iloc[-1])
else:
    stats["timestamp_min"] = None
    stats["timestamp_max"] = None

stats, os.path.exists(output_path), output_path


({'rows_before': 30112,
  'rows_dropped': 9000,
  'rows_after': 21112,
  'columns': ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'],
  'timestamp_min': '2012-01-01 16:00:00+07:00',
  'timestamp_max': '2021-08-20 04:00:00+07:00'},
 True,
 '/btc_4h_minus_last9000.csv')

#Merging both dataset

menggbungkan 70% data historis dengan 30% data hasil scrapping


In [ ]:
import pandas as pd


old_path = "/btc_4h_minus_last9000.csv"      # dataset lama (setelah drop 6000 terakhir)
new_path = "/btcusd_bitstamp_4h_9000.csv"    # 6000 candle terbaru (hasil scraping)
out_path = "/btc_4h_merged.csv"


def normalize_columns(df):
    df.columns = [c.strip().lower() for c in df.columns]
    rename_map = {
        "timestamp": "Timestamp",
        "open": "Open",
        "high": "High",
        "low": "Low",
        "close": "Close",
        "volume": "Volume",
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
    final_cols = ["Timestamp", "Open", "High", "Low", "Close", "Volume"]
    df = df[[c for c in final_cols if c in df.columns]]
    return df


def to_wib_naive(ts_series):
    s = ts_series.astype(str)
    def _convert_one(x):
        try:
            dt = pd.to_datetime(x)
        except Exception:
            return pd.NaT
        if getattr(dt, "tz", None) is not None:
            return dt.tz_convert("Asia/Jakarta").tz_localize(None)
        return dt
    return s.apply(_convert_one)


df_old = pd.read_csv(old_path, dtype={"Timestamp": "string"})
df_new = pd.read_csv(new_path, dtype={"Timestamp": "string"})

df_old = normalize_columns(df_old)
df_new = normalize_columns(df_new)

df_old["Timestamp"] = to_wib_naive(df_old["Timestamp"])
df_new["Timestamp"] = to_wib_naive(df_new["Timestamp"])

for df in (df_old, df_new):
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")


df_merged = pd.concat([df_old, df_new], ignore_index=True)

df_merged.drop_duplicates(subset=["Timestamp"], keep="last", inplace=True)

df_merged.sort_values("Timestamp", inplace=True)


df_merged.to_csv(out_path, index=False)

print("Output disimpan ke:", out_path)
print("Total baris:", len(df_merged))
if not df_merged.empty:
    print("Range waktu:", df_merged["Timestamp"].iloc[0], "→", df_merged["Timestamp"].iloc[-1])

print(df_merged["Timestamp"].diff().value_counts().head())


Output disimpan ke: /btc_4h_merged.csv
Total baris: 30112
Range waktu: 2012-01-01 16:00:00 → 2025-11-07 19:00:00
Timestamp
0 days 04:00:00     30110
40 days 19:00:00        1
Name: count, dtype: int64
